# Protocol 3: Functional characterization of replicating clusters

Protocol 3 demonstrates how to characterize functional gene sets contributing to cell type identity. In this section, we will focus on the characterization of inhibitory neuron subclasses as provided by the BICCN. The BICCN has shown that subclasses are strongly replicable across datasets and provided marker genes that are specific to each subclass. MetaNeighbor can be used to further quantify which pathways contribute to the subclasses' unique biological properties.

## Step 1 - Creation of biologically relevant gene sets

1. To compute the functional characterization of clusters, we first need an ensemble of gene sets sampling relevant biological pathways. In this protocol we will consider the Gene Ontology (GO) annotation for mouse. The scripts used to build up-to-date gene sets are here XXX, gene sets can be downloaded directly here XXX.  

In [ ]:
go_sets = readRDS("scripts/go_mouse.rds")

Gene sets are stored as a named list, each element of the list corresponds to a gene set and contains a vector of gene symbols.

2. Then we load our dataset containing inhibitory neurons from the BICCN. The scripts used to build the dataset can be found here XXX, the dataset can be downloaded here XXX.

In [ ]:
library(SingleCellExperiment)
biccn_gaba = readRDS("biccn_gaba.rds")
dim(biccn_gaba)

3. Next we restrict our gene sets to genes that are present in the dataset. We then filter gene sets to obtain gene sets of meaningful size: large enough to learn expression profiles (> 10), small enough to represent specific biological function or processes (< 100).

In [ ]:
known_genes = rownames(biccn_gaba)
go_sets = lapply(go_sets, function(gene_set) { gene_set[gene_set %in% known_genes] })
min_size = 10
max_size = 100
go_set_size = sapply(go_sets, length)
go_sets = go_sets[go_set_size >= min_size & go_set_size <= max_size]
length(go_sets)

## Step 2: Functional characterization with supervised MetaNeighbor

4. Once the gene set list is ready, we run the supervised "MetaNeighbor" function. Its inputs are similar to "MetaNeighborUS", but it assumes that cell types have already been matched across datasets (i.e., they have identical names). Here we will use joint BICCN subclasses, for which names have been normalized across datasets ("Pvalb", "Sst", "Sst Chodl", "Vip", "Lamp5", "Sncg"). Note that, because we are testing close to 6,500 gene sets, this step is expected to take a long time for large datasets. We recommend using this function inside a script and always save results to a file as soon as it's done.

In [ ]:
#library(MetaNeighbor)
devtools::load_all("~/projects/metaneighbor/MetaNeighbor")
if (FALSE) {
system.time({
aurocs = MetaNeighbor(dat = biccn_gaba,
                      experiment_labels = biccn_gaba$study_id,
                      celltype_labels = biccn_gaba$joint_subclass_label,
                      genesets = go_sets,
                      fast_version = TRUE, bplot = FALSE, batch_size = 50)
})
write.table(aurocs, "functional_aurocs.txt")
}

Later, results can be retrieved with the "read.table" function:

In [ ]:
aurocs = read.table("functional_aurocs.txt")

5. We use the "plotBPlot" function on the first 100 gene sets to rapidly visualize how replicability depends on gene sets.

In [ ]:
plotBPlot(head(aurocs, 100))

We note that, on average, gene sets contribute moderately to replicability (AUROC ~ 0.7), numerous gene sets have a performance close to random (AUROC ~ 0.5 - 0.6) and some gene sets have exceedingly high performance (AUROC > 0.8).

6. To focus on gene sets that contribute highly to specificity, we create a summary table containing, for each gene set, cell type specific AUROCs, average AUROC across all cell types and gene set size.

In [ ]:
gs_size = sapply(go_sets, length)
aurocs_df = as.data.frame(aurocs)
aurocs_df$average_auroc = rowMeans(aurocs)
aurocs_df$gs_size = gs_size[rownames(aurocs)]

We then order gene set by AUROC and look at top scoring gene sets:

In [ ]:
head(aurocs_df[order(aurocs_df$average_auroc, decreasing = TRUE),],10)

Without surprise, replicability is mainly driven by gene sets related to neuronal functions that are immediately relevant to the physiology of inhibitory neurons, such as "glutamate receptor signaling pathway", "regulation of synaptic transmission, glutamatergic", or "chemical synaptic transmission, postsynaptic". Note that most of the top scoring gene sets have a large number of genes. To obtain even more specific biological function, we can further filter gene sets that have fewer than 20 genes.

In [ ]:
small_sets = aurocs_df[aurocs_df$gs_size < 20,]
head(small_sets[order(small_sets$average_auroc, decreasing = TRUE),],10)

Again, the top scoring gene sets are dominated by biological functions immediately relevant to inhibitory neuron physiology, such as "ionotropic glutamate receptor signaling pathway", "positive regulation of synaptic transmission, GABAergic", or "GABA-A receptor complex".

7. To understand how individual genes contribute to gene set performance, we use the "plotDotPlot" function, which while show the expression of all genes in a gene set of interest, averaged over all datasets.

In [ ]:
library(tidyverse)
plotDotPlot(dat = biccn_gaba,
            experiment_labels = biccn_gaba$study_id,
            celltype_labels = biccn_gaba$joint_subclass_label,
            gene_set = go_sets[["GO:0007215|glutamate receptor signaling pathway|BP"]])
plotDotPlot(dat = biccn_gaba,
            experiment_labels = biccn_gaba$study_id,
            celltype_labels = biccn_gaba$joint_subclass_label,
            gene_set = go_sets[["GO:1902711|GABA-A receptor complex|CC"]])

High scoring gene sets are characterized by the differential usage of genes from a given gene set. For example, when looking at the GABA-A receptor complex composition, Lamp5 will preferentially use the Gabrb2 and Gabrg3 receptors, Pvalb the Gabra1 receptor, and Sst Chodl the Gabra2, Gabrb1 and Gabrg1 receptors.

Once the overall replicability of clusters has been established with unsupervised MetaNeighbor (as in Protocols 1 and 2), supervised MetaNeighbor enables the functional interpretation of the biology contributing to each cell type's unique identity.